In [5]:
!pip install pdfplumber langdetect sentence-transformers transformers sentencepiece -q

In [27]:
# ============================
# 1. Gerekli Kütüphaneler
# ============================
import re
import pdfplumber
from langdetect import detect
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

# ============================
# 2. Anahtar Kelimeler ve Ağırlıklar (Kısaca)
# ============================
criteria_labels = {
    "iş_deneyimi": ["İş Deneyimi", "Work Experience", "Career History", "Experience"],
    "egitim": ["Eğitim", "Education", "Academic Background", "University"],
    "teknik_beceriler": ["Teknik Beceriler", "Technical Skills", "Skills"],
    "ozet": ["Özet", "Hakkımda", "Summary", "Profile", "Objective"],
    "liderlik": ["Liderlik", "Leadership", "Organization Experience"],
    "sertifikalar": ["Sertifikalar", "Certificates", "Certifications", "Courses"],
    "iletisim": ["İletişim", "Contact", "Phone", "Email"],
    "portfolyo": ["Portfolyo", "Portfolio", "GitHub", "Website"],
    "diller": ["Diller", "Languages"],
    "referanslar": ["Referanslar", "References"]
}

criteria_weights = {
    "iş_deneyimi": 25,
    "egitim": 15,
    "teknik_beceriler": 15,
    "ozet": 10,
    "liderlik": 10,
    "sertifikalar": 10,
    "iletisim": 5,
    "portfolyo": 5,
    "diller": 3,
    "referanslar": 2
}

# ============================
# 3. Modellerin Yüklenmesi
# ============================
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')  # Hızlı, çok dilli
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

def get_ner_pipeline(language_code):
    if language_code == "tr":
        return pipeline("ner", model="savasy/bert-base-turkish-ner-cased", aggregation_strategy="simple")
    else:
        return pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

# ============================
# 4. PDF Metin Çıkarma
# ============================
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# ============================
# 5. Hibrit Analiz Fonksiyonu
# ============================
def analyze_cv(text):
    lang = detect(text)
    text_lower = text.lower()
    total_score = 0
    strengths, weaknesses = [], []

    for section, labels in criteria_labels.items():
        max_score = 0

        for kw in labels:
            if kw.lower() in text_lower:
                max_score = 1.0
                break

        if max_score < 1.0:
            embeddings1 = semantic_model.encode(text, convert_to_tensor=True)
            embeddings2 = semantic_model.encode(" ".join(labels), convert_to_tensor=True)
            sim = float(util.cos_sim(embeddings1, embeddings2))
            if sim > 0.4:
                max_score = max(max_score, sim)

        if max_score < 0.4:
            try:
                result = classifier(text[:512], candidate_labels=labels, multi_label=True)
                zs_score = max(result["scores"]) if "scores" in result else 0
                max_score = max(max_score, zs_score)
            except:
                pass

        section_score = round(criteria_weights[section] * max_score, 2)
        total_score += section_score

        if max_score >= 0.6:
            strengths.append(f"{section.replace('_', ' ').title()} bölümü mevcut ve yeterli.")
        elif max_score >= 0.4:
            weaknesses.append(f"{section.replace('_', ' ').title()} bölümü zayıf.")
        else:
            weaknesses.append(f"{section.replace('_', ' ').title()} bölümü eksik.")

    total_score = round(total_score, 2)
    return {
        "language": lang,
        "total_score": total_score,
        "strengths": strengths,
        "weaknesses": weaknesses
    }

# ============================
# 6. Rapor Görselleştirme
# ============================
def color_for_score(score):
    if score >= 75:
        return "green"
    elif score >= 50:
        return "orange"
    else:
        return "red"

def lang_display_name(code):
    return {"tr": "Türkçe", "en": "English"}.get(code, code)

def display_report(report):
    display(Markdown(f"## 📊 CV Analiz Raporu"))
    display(Markdown(f"**Dil:** {lang_display_name(report['language'])}"))
    display(Markdown("---"))

    score = report['total_score']
    color = color_for_score(score)
    score_html = f"""
    <div style='font-size:48px; font-weight:bold; color:{color}; text-align:center;'>
        {score} / 100
    </div>
    """
    display(HTML(score_html))

    strengths_md = "### ✅ Güçlü Yönler\n"
    if report['strengths']:
        for s in report['strengths']:
            strengths_md += f"- {s}\n"
    else:
        strengths_md += "- Belirgin güçlü yön bulunamadı.\n"
    display(Markdown(strengths_md))

    weaknesses_md = "### ⚠️ Zayıf Yönler\n"
    if report['weaknesses']:
        for w in report['weaknesses']:
            weaknesses_md += f"- {w}\n"
    else:
        weaknesses_md += "- Belirgin zayıf yön bulunamadı.\n"
    display(Markdown(weaknesses_md))

# ============================
# 7. PDF Yükleme ve Çalıştırma
# ============================
def manual_pdf_upload():
    try:
        from google.colab import files
        print("Lütfen analiz edilecek CV PDF dosyasını yükleyin:")
        uploaded = files.upload()
        pdf_path = list(uploaded.keys())[0]
        print(f"'{pdf_path}' başarıyla yüklendi.")
        return pdf_path
    except ImportError:
        pdf_path = input("PDF dosyasının tam yolunu girin: ")
        return pdf_path

def run_interactive_report():
    pdf_path = manual_pdf_upload()
    cv_text = extract_text_from_pdf(pdf_path)
    report = analyze_cv(cv_text)
    display_report(report)

# ============================
# 8. Çalıştırma
# ============================
if __name__ == "__main__":
    run_interactive_report()


Device set to use cpu


Lütfen analiz edilecek CV PDF dosyasını yükleyin:


Saving Curriculum Vitae - Meriç Özcan.pdf to Curriculum Vitae - Meriç Özcan (6).pdf
'Curriculum Vitae - Meriç Özcan (6).pdf' başarıyla yüklendi.


## 📊 CV Analiz Raporu

**Dil:** English

---

### ✅ Güçlü Yönler
- Iş Deneyimi bölümü mevcut ve yeterli.
- Egitim bölümü mevcut ve yeterli.
- Teknik Beceriler bölümü mevcut ve yeterli.
- Liderlik bölümü mevcut ve yeterli.
- Iletisim bölümü mevcut ve yeterli.
- Portfolyo bölümü mevcut ve yeterli.
- Diller bölümü mevcut ve yeterli.
- Referanslar bölümü mevcut ve yeterli.


### ⚠️ Zayıf Yönler
- Ozet bölümü zayıf.
- Sertifikalar bölümü eksik.
